In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd Data

/content/drive/MyDrive/Data


In [5]:
import pandas as pd
non_PRC = pd.read_csv('eye_open/eye_open.csv')
PRC = pd.read_csv('eye_open/eye_open_par.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [17]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [7]:
warnings.filterwarnings(action='ignore')

In [8]:
score = clinical_assessment.iloc[:,9]

In [9]:
score = score.dropna()
del score[0]
del score[15]

In [10]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-paretic Side

In [11]:
df_non = pd.DataFrame(non_PRC)

In [12]:
y = pd.DataFrame(y)

In [13]:
non = pd.concat([y,df_non],axis=1)
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34',
       'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43'],
      dtype='object')

In [14]:
X = non.loc[:,['PC2','PC13','PC15','PC18','PC20','PC34']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### logistic regression

In [15]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Test Accuracy:", accuracy_lr)

# f1-score 계산
f1_lr = f1_score(y_test, y_pred_lr)
print("f1-score:", f1_lr)

# auc 계산
auc_lr = roc_auc_score(y_test,y_pred_lr)
print("AUC:",auc_lr)

# 분류 보고서 출력
classification_rep_lr = classification_report(y_test, y_pred_lr)
print("Classification Report:\n", classification_rep_lr)

Confusion Matrix:
 [[5 1]
 [3 9]]
Test Accuracy: 0.7777777777777778
f1-score: 0.8181818181818182
AUC: 0.7916666666666667
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.83      0.71         6
           1       0.90      0.75      0.82        12

    accuracy                           0.78        18
   macro avg       0.76      0.79      0.77        18
weighted avg       0.81      0.78      0.78        18



### Random Forest

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.8333999999999999
Average Precision: 0.8168
Average F1-score: 0.8914
Average AUC: 0.7666000000000001


### SVM Linear

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.85
Average Precision: 0.85
Average F1-score: 0.9048
Average AUC: 0.9334


### SVM Poly

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.5666
Average Precision: 0.65
Average F1-score: 0.6848000000000001
Average AUC: 0.3


### XGBoost

In [21]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7666000000000001
Average Precision: 0.7834
Average F1-score: 0.8314
Average AUC: 0.8333999999999999


## paretic-Side

In [22]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [23]:
import random
random.seed(1)
X = par.loc[:,['PC2','PC5','PC15','PC18','PC25','PC38']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### logistic regression

In [24]:
lr_model = LogisticRegression()

lr_model.fit(X_train, y_train)

y_pred_par_lr = lr_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred_par_lr)
print("Confusion Matrix:\n", conf_matrix)
# 정확도 계산
accuracy_lr_ps = accuracy_score(y_test, y_pred_par_lr)
print("Test Accuracy:", accuracy_lr_ps)

# f1-score 계산
f1_lr_ps = f1_score(y_test, y_pred_par_lr)
print("f1-score:", f1_lr_ps)

# AUC 계산
auc_lr_ps = roc_auc_score(y_test,y_pred_par_lr)
print("AUC:",auc_lr_ps)


classification_rep_lr = classification_report(y_test, y_pred_par_lr)
print("Classification Report:\n", classification_rep_lr)

Confusion Matrix:
 [[4 2]
 [3 9]]
Test Accuracy: 0.7222222222222222
f1-score: 0.7826086956521738
AUC: 0.7083333333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.82      0.75      0.78        12

    accuracy                           0.72        18
   macro avg       0.69      0.71      0.70        18
weighted avg       0.74      0.72      0.73        18



### paretic RandomForest

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6666000000000001
Average Precision: 0.75
Average F1-score: 0.7647999999999999
Average AUC: 0.7333999999999999


### paretic SVM Linear

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.5668
Average Precision: 0.8333999999999999
Average F1-score: 0.6336
Average AUC: 0.5668


### paretic SVM Poly

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.6668
Average Precision: 0.7167999999999999
Average F1-score: 0.7847999999999999
Average AUC: 0.8


### paretic XGBoost

In [28]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Average Accuracy: 0.7834
Average Precision: 0.9
Average F1-score: 0.8268000000000001
Average AUC: 0.7333999999999999
